In [1]:
import time
import MetaTrader5 as mt5
import pandas as pd
from random import seed
from random import randint
import random
import datetime

In [2]:
mt5.initialize()

True

In [3]:
symbol = "EURUSD"
def open_order(symbol,lot,buysell):
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 20
    if buysell=='buy':
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "deviation": deviation,
            "magic": 234000,
            "comment": "python script open",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK,
        }
    elif buysell=='sell':
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "deviation": deviation,
            "magic": 234000,
            "comment": "python script open",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK,
        }
    # envoie une demande de trading
    result = mt5.order_send(request)
    return result
def close_position(position):

    tick = mt5.symbol_info_tick(position.request.symbol)

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position.order,
        "symbol": position.request.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.request.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.request.type == 1 else tick.bid,  
        "deviation": 20,
        "magic": 100,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    result = mt5.order_send(request)
    return result


In [4]:
class Position :
    def __init__(self,open_price,position,takeprofit,lot_size,rg,mark,stop):

        self.lot_size=lot_size
        self.position=position
        self.stop=stop
        self.takeprofit=takeprofit
        self.rg=rg
        self.mark=mark
        self.shared=1
        
        
        
        if position=='buy':
            self.mul=-1
            self.order=open_order("EURUSD",self.lot_size,"buy")
            
        if position =='sell':
            self.mul=1
            self.order=open_order("EURUSD",self.lot_size,"sell")
        self.pl=0.0
        self.calculatedpl=0
    def close_position(self):
        close_position(self.order)



In [7]:
open_positions=[]

lot_size=0.01

for i in range(20):
    open_positions.append([])
while True :
    now = datetime.datetime.now()
    price=mt5.symbol_info_tick("EURUSD").bid
    for ps in range(20):
        s=random.randint(0,1)
        if open_positions[ps]==[] and (now.hour<=17) and (now.hour>=1):
            if s==0:
                takeprofit=price+0.00165
                stop=price-0.0015
                open_positions[ps].append(Position(price,"buy",takeprofit,lot_size,1,1,stop))
            elif s==1:
                takeprofit=price-0.00165
                stop=price+0.0015
                open_positions[ps].append(Position(price,"sell",takeprofit,lot_size,1,1,stop))
    for z in range(len(open_positions)):
        for j in range(len(open_positions[z])) :
            if open_positions[z][j].position=='buy':
                if open_positions[z][j].rg==1 and open_positions[z][j].mark==1 :
                    if price>=open_positions[z][j].takeprofit:
                        open_positions[z][j].close_position()
                        open_positions[z]=[]
                    elif price<=open_positions[z][j].stop :
                        open_positions[z][j].mark=2
                        stop=price+0.0015
                        takeprofit=price-0.00165
                        open_positions[z].append(Position(price,'sell',takeprofit,open_positions[z][j].lot_size*8,2,2,stop))
                elif open_positions[z][j].rg==2 and open_positions[z][j].mark==2 :
                    if price>=open_positions[z][j].takeprofit :
                        for p1 in range(len(open_positions[z])):
                            open_positions[z][p1].close_position()
                        open_positions[z]=[]
                        
                    elif price<=open_positions[z][j].stop:
                        for p1 in range(len(open_positions[z])):
                            open_positions[z][p1].close_position()
                        open_positions[z]=[]
                        
                
                    ######################################################################################################
            
            elif open_positions[z][j].position=='sell':
                if open_positions[z][j].rg==1 and open_positions[z][j].mark==1 :
                    if price<=open_positions[z][j].takeprofit:
                        open_positions[z][j].close_position()
                        open_positions[z]=[]
                        break
                    elif price>=open_positions[z][j].stop :
                        open_positions[z][j].mark=2
                        stop=price-0.0015
                        takeprofit=price+0.00165
                        open_positions[z].append(Position(price,'buy',takeprofit,open_positions[z][j].lot_size*8,2,2,stop))
                elif open_positions[z][j].rg==2 and open_positions[z][j].mark==2 :
                    if price<=open_positions[z][j].takeprofit :
                        for p1 in range(len(open_positions[z])):
                            open_positions[z][p1].close_position()
                        open_positions[z]=[]
                        break
                    elif price>=open_positions[z][j].stop:
                        for p1 in range(len(open_positions[z])):
                            open_positions[z][p1].close_position()
                        open_positions[z]=[]

KeyboardInterrupt: 

In [6]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [15]:
print(mt5.symbol_info_tick("BTCUSD"))

Tick(time=1642977147, bid=35149.5, ask=35163.02, last=0.0, volume=0, time_msc=1642977147321, flags=6, volume_real=0.0)


In [26]:
ss=open_order("BTCUSD",0.01,"buy")
print(ss)

OrderSendResult(retcode=10009, deal=3594717, order=5498254, volume=0.01, price=35187.55, bid=0.0, ask=0.0, comment='Request executed', request_id=1, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='BTCUSD', volume=0.01, price=35185.45, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


In [27]:
ss=open_order("BTCUSD",0.01,"sell")
print(ss)

OrderSendResult(retcode=10009, deal=3594718, order=5498255, volume=0.01, price=35279.46, bid=0.0, ask=0.0, comment='Request executed', request_id=2, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='BTCUSD', volume=0.01, price=35294.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


In [58]:
ss=open_order("BTCUSD",0.01,"buy")
print(ss)

OrderSendResult(retcode=10009, deal=3594727, order=5498263, volume=0.01, price=35364.28, bid=0.0, ask=0.0, comment='Request executed', request_id=7, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='BTCUSD', volume=0.01, price=35364.28, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


In [104]:
ss=open_order("BTCUSD",0.01,"sell")
print(ss)

OrderSendResult(retcode=10009, deal=3594793, order=5498398, volume=0.01, price=35479.63, bid=0.0, ask=0.0, comment='Request executed', request_id=19, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='BTCUSD', volume=0.01, price=35493.18, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


In [105]:
close_position(ss)

OrderSendResult(retcode=10009, deal=3594794, order=5498399, volume=0.01, price=35491.93, bid=0.0, ask=0.0, comment='Request executed', request_id=20, retcode_external=0, request=TradeRequest(action=1, magic=100, order=0, symbol='BTCUSD', volume=0.01, price=35491.93, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script close', position=5498398, position_by=0))